First lets install the correct packages for GPT3.  We are already in the conda environment from jupyter.

First lets install pytorch.

In [4]:
conda install -c conda-forge cudatoolkit

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Popin\anaconda3\envs\jupyter_gpt

  added / updated specs:
    - cudatoolkit


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.10.8  |       h5b45459_0         176 KB  conda-forge
    certifi-2021.10.8          |   py37h03978a9_1         145 KB  conda-forge
    cudatoolkit-11.6.0         |      hc0ea762_10       965.1 MB  conda-forge
    openssl-1.1.1l             |       h8ffe710_0         5.7 MB  conda-forge
    ------------------------------------------------------------
                                           Total:       971.1 MB

The following NEW packages will be INSTALLED:

  cudatoolkit        conda-forge/win-64::cudatoolkit-11.6.0-hc0ea762_10

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates

In [1]:
conda install -c fastchan cudatoolkit

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Popin\anaconda3\envs\jupyter_gpt

  added / updated specs:
    - cudatoolkit


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cudatoolkit-11.1.1         |       heb2d755_9        1.20 GB  fastchan
    ------------------------------------------------------------
                                           Total:        1.20 GB

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2021.10.~ --> pkgs/main::ca-certificates-2022.2.1-haa95532_0
  certifi            conda-forge::certifi-2021.10.8-py37h0~ --> pkgs/main::certifi-2021.10.8-py37haa95532_2
  openssl            conda-forge::openssl-1.1.1l-h8ffe710_0 --> pkgs/main::openssl-1.1.1m-h2bbff1b_0

The following packages will be SUPERSEDED by a higher-priority channel:

  cudatoolkit        conda-forge::cudatool

In [2]:
!conda install pytorch torchvision torchaudio cudatoolkit=11.1 -c pytorch -c nvidia -y

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Popin\anaconda3\envs\jupyter_gpt

  added / updated specs:
    - cudatoolkit=11.1
    - pytorch
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libuv-1.40.0               |       he774522_0         255 KB
    pillow-9.0.1               |   py37hdc2b20a_0         916 KB
    pytorch-1.11.0             |      py3.7_cpu_0       129.9 MB  pytorch
    pytorch-mutex-1.0          |              cpu           3 KB  pytorch
    torchaudio-0.11.0          |         py37_cpu         2.9 MB  pytorch
    torchvision-0.12.0         |         py37_cpu         7.4 MB  pytorch
    ------------------------------------------------------------
                                           Total:       141.4 MB

The following NEW packages will be INSTALLED:

  freetype           pkgs

Now lets install HuggingFace.  It makes using popular Tranformers MUCH easier.

In [3]:
!conda install -c huggingface transformers -y

Solving environment: ...working... done

# All requested packages already installed.



Lets import the needed packages now

In [4]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import torch

Now lets get the model.  We can either run the 1.3 billion paramater model or the 2.7 billion parameter model. Lets do the 2.7B model, which is "EleutherAI/gpt-neo-2.7B".  The 1.3B model is "EleutherAI/gpt-neo-1.3B"

In [5]:
model_name = "EleutherAI/gpt-neo-2.7B"
model = GPTNeoForCausalLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

This model can be ran on a GPU, but does not have to be. The 2.7B model takes slightly less than 13 GB of Vram.  The 1.3B model takes slighly less than 7.5GB of Vram.  The model will be placed on the GPU if there is one and if there is enough Vram.

Lets install pynvml to take a look at how much VRAM we have.

In [6]:
!pip install pynvml

In [30]:
free_vram = 6
if torch.cuda.is_available():
    from pynvml import *
    nvmlInit()
    h = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(h)
    free_vram = info.free/1048576000
    print("There is a GPU with " + str(free_vram) + "GB of free VRAM")

In [40]:
if model_name == "EleutherAI/gpt-neo-2.7B" and free_vram>13.5:
    use_cuda = True
    model.to("cuda:0")
elif model_name == "EleutherAI/gpt-neo-1.3B" and free_vram>5:
    use_cuda = True
    model.to("cuda:0")
else:
    use_cuda = False

Now we need to load the tokenizer to prepare the input for GPT3

In [41]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

We are almost done. At this point we need to decide what prompt we need to decide what prompt we want the model to continue, as well a how long we want the generated output to be.

In [42]:
prompt = str(input("Please enter a prompt: "))

KeyboardInterrupt: Interrupted by user

In [ ]:
output_length = int(input("How long should the generated output be? "))

We now need to tokenize the input prompt to prepare it for use with the model.  If we are using a GPU we will put it on the GPU as well.

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
if use_cuda:
    input_ids = input_ids.cuda()

In [ ]:
gen_tokens = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=output_length)

In [ ]:
gen_text = tokenizer.batch_decode(gen_tokens)[0]
print(gen_text)